# AOC 2023

Welcome to the Advent of Code 2023 !

## Basic configuration



In [ ]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [ ]:
import os

# replace by your login session cookie
os.environ[
    "AOC_SESSION"
] = ""  # your login session cookie

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [ ]:
import numpy as np
from tqdm import tqdm
import json
import typing as typ
from collections import Counter, defaultdict, deque, ChainMap
import math
import itertools
import re
import regex
import string
import matplotlib.pyplot as plt

## Day 8

https://adventofcode.com/2023/day/8

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=8)
instructions, content = puzzle.input_data.split("\n\n")

In [ ]:
network = dict()
for line in content.split("\n"): 
    start, left, right = re.findall("\w+", line)
    network[start]={"L":left, "R":right}

In [ ]:
instructions, network

### Part 1

In [ ]:
def loop_on_condition(current, condition:typ.Callable[str, bool])->int: 
    for count, inst in enumerate(itertools.cycle(instructions)): 
            current = network[current][inst] 
            if condition(current): 
                return count + 1

In [ ]:
answ = loop_on_condition("AAA",lambda x : x == "ZZZ") 
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
starts = [key for key in network.keys() if key[-1]=="A"]
cycle_lengths = [loop_on_condition(start,lambda x : x[-1] == "Z") for start in starts]

In [ ]:
cycle_lengths

In [ ]:
answ = math.lcm(*cycle_lengths)
answ

In [ ]:
puzzle.answer_b = answ

## Day 7

https://adventofcode.com/2023/day/7

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=7)
content = puzzle.input_data.split("\n")
content

In [ ]:
map_card = {"T":10,"J":11,"Q":12,"K":13,"A":14}
hands = [[[int(map_card.get(card, card)) for card in line.split()[0]], Counter(line.split()[0]), int(line.split()[1])] for line in content]
hands

### Part 1

In [ ]:
def winnings(hands:typ.List[typ.List[typ.Any]])->int:

    def rank(x:int,y:int)->int: 
        return (x+1)*y
        
    def sorting_fnc(hand:typ.List[typ.Any])->typ.Tuple[typ.List[int],typ.List[int]]:
        return sorted(hand[1].values(), reverse=True), hand[0]

    return sum(rank(i,hand[2]) for i,hand in enumerate(sorted(hands, key=sorting_fnc)))

In [ ]:
answ = winnings(hands)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
new_map_card = {"T":10,"J":0,"Q":12,"K":13,"A":14}
new_hands = []
for line in content: 
    hand, bid = line.split()
    hand_count = Counter(hand) 
    new_hand = [int(new_map_card.get(card, card)) for card in hand]

    j_count = hand_count["J"]
    if  j_count != 5: 
        hand_count["J"]=0
        mc = hand_count.most_common()[0]
        hand_count[mc[0]]= hand_count[mc[0]] + j_count
        
    new_hands.append([new_hand, hand_count, int(bid)])

In [ ]:
new_hands

In [ ]:
answ = winnings(new_hands)
answ

In [ ]:
puzzle.answer_b = answ

## Day 6

https://adventofcode.com/2023/day/6

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=6)
content = puzzle.input_data.split("\n")
content

In [ ]:
def parse_1(s:str)->typ.List[int]: 
    return list(map(int,re.findall("\d+", s)))
times = parse_1(content[0])
distances = parse_1(content[1])

### Part 1

In [ ]:
answ = math.prod(sum((t*(tot_time-t))>record_dist for t in range(tot_time)) for tot_time, record_dist in zip(times, distances))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def parse_2(s:str)->int:
    return int("".join(re.findall("\d+", s)))
time = parse_2(content[0])
dist = parse_2(content[1])

In [ ]:
answ = sum((t*(time-t))>dist for t in tqdm((range(time))))
answ

In [ ]:
puzzle.answer_b = answ

## Day 5

https://adventofcode.com/2023/day/5

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=5)
content =[line.split(":") for line in puzzle.input_data.split("\n\n")]
#content

In [ ]:
seeds = list(map(int, re.findall(r"\d+", content[0][1])))

In [ ]:
mappings = dict()
for line in content[1:]: 
    source_name, dest_name = re.findall(r"(\w+)-to-(\w+) map", line[0])[0]
    mappings[source_name] = {"dest": dest_name, 
                             "mapping": [list(map(int, re.findall(r"\d+", l))) for l in line[1].strip().split("\n")]}

In [ ]:
mappings

### Part 1

In [ ]:
def apply_maps(number:int,cur_source:str)->int: 
    if cur_source == "location":
        return number
    cur_map = mappings[cur_source]
    new_source = cur_map["dest"]
    for dest_start, source_start, range_len in cur_map["mapping"]: 
        if source_start <= number < source_start + range_len: 
            new_number = dest_start + number - source_start
            return apply_maps(new_number, new_source)
    return apply_maps(number, new_source)

In [ ]:
answ = min(apply_maps(seed_num, "seed") for seed_num in seeds)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
reverse_mappings = dict()
for line in content[1:]: 
    source_name, dest_name = re.findall(r"(\w+)-to-(\w+) map", line[0])[0]
    reverse_mappings[dest_name] = {"source": source_name, 
                             "mapping": [list(map(int, re.findall(r"\d+", l))) for l in line[1].strip().split("\n")]}

In [ ]:
reverse_mappings

In [ ]:
def is_seed(number:int)->bool: 
    for start,range in zip(seeds[::2],seeds[1::2]):
        if start <= number < start+range: 
            return True
    return False

In [ ]:
def reverse_maps(number:int, cur_dest:str)->int: 
    if cur_dest == "seed": 
        return number
    cur_map = reverse_mappings[cur_dest]
    new_dest = cur_map["source"]
    for dest_start, source_start, range_len in cur_map["mapping"]: 
        if dest_start <= number < dest_start + range_len: 
            new_number = source_start + number - dest_start 
            return reverse_maps(new_number, new_dest)
    return reverse_maps(number, new_dest) 

In [ ]:
from itertools import count
for loc in tqdm(count()): 
    if is_seed(reverse_maps(loc, "location")):
        break
loc

In [ ]:
puzzle.answer_b = loc

## Day 4

https://adventofcode.com/2023/day/4

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=4)
content =[re.split(r":|\|", line) for line in puzzle.input_data.split("\n")]
content

### Part 1

In [ ]:
n_winnings = [(sum(x in list(map(int,ours.strip().split())) for x in list(map(int,winning.strip().split()))) ) for _,winning,ours in content]

In [ ]:
points = sum(2**(n-1) for n in n_winnings if n!=0)

In [ ]:
points

In [ ]:
puzzle.answer_a = points

### Part 2

In [ ]:
copies = {id+1 : 1 for id in range(len(content))}

In [ ]:
for cur_id, n_wins in enumerate(n_winnings): 
    for next_id in range(cur_id+1, cur_id+n_wins+1): 
        copies[next_id] += copies[cur_id]

In [ ]:
answ = sum(copies.values())
answ

In [ ]:
puzzle.answer_b = answ

## Day 3

https://adventofcode.com/2023/day/3

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=3)
content = puzzle.input_data.split("\n")
content

### Part 1

In [ ]:
def check_neighbours(array:typ.List[str], row:int, col:int)->bool:

    def get_neighbours()->typ.Tuple[str,str,str,str,str,str,str,str]:
        top = array[row-1][col] if row != 0 else "."
        bottom = array[row+1][col] if row != len(array)-1 else "." 
        left = array[row][col-1] if col != 0 else "."
        right = array[row][col+1] if col != len(array[0])-1 else "."
        
        tl = array[row-1][col-1] if row != 0 and col != 0 else "."
        tr = array[row-1][col+1] if row != 0 and col != len(array[0])-1 else "."
        bl = array[row+1][col-1] if row != len(array)-1 and col != 0 else "."
        br = array[row+1][col+1] if row != len(array)-1 and col != len(array[0])-1 else "."
        
        return top, bottom, left, right, tl, tr, bl, br
    
    return any(n in string.punctuation.replace(".","") for n in get_all_neighbours())


In [ ]:
parts = [(number, row) 
    for row, row_numbers in enumerate(re.finditer(r"\d+", line) for line in content) 
    for number in row_numbers
    if any(check_neighbours(content, row, col) for col in range(number.span()[0],number.span()[1]))
]

answ = sum(int(part[0].group()) for part in parts)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def find_gears(array:typ.List[str], row:int, col:int)->typ.List[typ.Tuple[int,int]]: 
    
    def get_neighbours_pos()->typ.List[typ.Tuple[int,int]]: 
        neighbours = []
        if col != 0: 
            neighbours.append((row, col-1))
        if col != len(array[0])-1: 
            neighbours.append((row, col+1))
        if row != 0: 
            neighbours.append((row-1, col))
            if col != 0: 
                neighbours.append((row-1, col-1))
            if col != len(array[0])-1: 
                neighbours.append((row-1, col+1))
        if row != len(array)-1: 
            neighbours.append((row+1,col))
            if col != 0: 
                neighbours.append((row+1, col-1))
            if col != len(array[0])-1: 
                neighbours.append((row+1, col+1))

        return neighbours

    return [n for n in get_neighbours_pos() if array[n[0]][n[1]] == "*"]
                                  

In [ ]:
gears = defaultdict(list)
for part in parts: 
    number = part[0]
    row = part[1]
    gear_positions = set()
    for col in range(number.span()[0],number.span()[1]): 
        gear_positions.update(find_gears(content, row, col))
    for gear_pos in gear_positions: 
        gears[gear_pos].append(int(number.group()))

In [ ]:
answ = sum(math.prod(gear_parts) for gear, gear_parts in gears.items() if len(gear_parts)==2) 
answ 

In [ ]:
puzzle.answer_b = answ

## Day 2

https://adventofcode.com/2023/day/2

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=2)

In [ ]:
content = puzzle.input_data.split("\n")
content

In [ ]:
def get_max_colors(line:str)->typ.Dict[str,int]: 
    max_colors = {"red":0,"blue":0,"green":0}
    for number, color in re.findall(r"(\d+) (\w+)", line.split(": ")[1]): 
        max_colors[color] = max(max_colors[color], int(number))
    return max_colors
    
all_max_colors = [get_max_colors(line) for line in content]

### Part 1

In [ ]:
target = {"red":12,"green":13,"blue":14}

In [ ]:
answ = sum((id+1)*all(max_colors[key]<=target[key] for key in max_colors.keys()) for id, max_colors in enumerate(all_max_colors))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = sum(math.prod(max_colors.values()) for max_colors in all_max_colors)
answ

In [ ]:
puzzle.answer_b = answ

## Day 1

https://adventofcode.com/2023/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=1)

In [ ]:
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
answ = sum(int(f"{x[0]}{x[-1]}") for x in [re.findall(r"\d", line) for line in content])
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
txt_digits = "one, two, three, four, five, six, seven, eight, nine".split(", ")

In [ ]:
mapping = [{k:int(v),v:int(v)} for k,v in zip(txt_digits,string.digits[1:])]
mapping = dict(ChainMap(*mapping))

In [ ]:
numbers = [regex.findall(fr"\d|{'|'.join(txt_digits)}", line, overlapped=True) for line in content]

In [ ]:
answ = sum(mapping[x[0]]*10 + mapping[x[-1]] for x in numbers)
answ

In [ ]:
puzzle.answer_b = answ